# Data Scrapping
* Nesse arquivo, utilizamos a biblioteca BeautifulSoup para extrair dados sobre imóveis do [Zap Imóveis](https://www.zapimoveis.com.br).
* O Zap imóveis usa uma proteção do [Cloudflare.com](https://www.cloudflare.com) para impedir que scripts tenham acesso às informações do site. Contornamos esse problema usando o pacote [cloudscraper](https://pypi.org/project/cloudscraper/), que simula o requester ter suporte à javascript (como um browser).
* Conseguimos importar com sucesso informações sobre 2889 apartamentos à venda na área de interesse, incluindo endereço, área, quantidade de dormitórios, quantidade de banheiros, quantidade de vagas de garagem, valor do condomínio, iptu, e preço de venda.

In [4]:
#importando as bibliotecas
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

Primeiramente, tentamos fazer o data scrapping da maneira convencional. Ou seja, tentamos usar a biblioteca requests para receber o HTML da página. Entretando, recebemos o seguinte erro:

**Access denied | www.zapimoveis.com.br used Cloudflare to restrict access**

In [2]:
# Load the webpage content
r = requests.get("https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro+zona-sul+flamengo/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.936822,-43.175702&transacao=Venda&tipo=Im%C3%B3vel%20usado&areaMaxima=35&tipoUnidade=Residencial,Apartamento&pagina=1")

# Convert to a beautiful soup object
soup = bs(r.content)

print(soup)

# Print out our html
#print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]--><!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]--><!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>Access denied | www.zapimoveis.com.br used Cloudflare to restrict access</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="noindex, nofollow" name="robots"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="/cdn-cgi/styles/main.css" id="cf_styles-css" media="screen,projection" rel="stylesheet" type="text/css"/>
</head>
<body>
<div id="cf-wrapper">
<div class="cf-alert cf-alert-error cf-cookie-error hidden" data-translate="enable_cookies" id="cookie-alert">Please enable cookies.</div>
<div class="p-0" id="cf-

Pesquisei sobre a maneira como o Cloudfare faz essa proteção: *Cloudflare's anti-bot page currently just checks if the client supports Javascript*. Para contornar esse problema, usamos o pacote [cloudscraper](https://pypi.org/project/cloudscraper/), que simula o requester ter suporte à javascript (como um browser).

Fiz um teste abaixo e consegui receber as informações dos anúncios do Zap com sucesso.

In [11]:
import cloudscraper

scraper = cloudscraper.create_scraper()  # returns a CloudScraper instance

#vamos testar o funcionamento com uma URL qualquer
url = 'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/avenida-oswaldo-cruz/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Botafogo,Avenida%20Oswaldo%20Cruz,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EBotafogo,-22.9398472,-43.175138&transacao=Venda&areaMaxima=100&areaMinima=0&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado&pagina=2'
request_body = scraper.get(url).text

soup = bs(request_body)

cards_all = soup.find_all("div", attrs={"class": "box--display-flex box--flex-column gutter-top-double gutter-left-double gutter-right-double gutter-bottom-double simple-card__box"})
#print(soup.prettify())

In [28]:
oswaldo_cruz = {
f'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/avenida-oswaldo-cruz/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Botafogo,Avenida%20Oswaldo%20Cruz,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EBotafogo,-22.9398472,-43.175138&transacao=Venda&areaMaxima=100&areaMinima=0&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 6,
f'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/avenida-oswaldo-cruz/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Botafogo,Avenida%20Oswaldo%20Cruz,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EBotafogo,-22.9398472,-43.175138&transacao=Venda&areaMaxima=130&areaMinima=101&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 4,
f'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/avenida-oswaldo-cruz/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Botafogo,Avenida%20Oswaldo%20Cruz,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EBotafogo,-22.9398472,-43.175138&transacao=Venda&areaMaxima=210&areaMinima=131&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 9,
f'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/avenida-oswaldo-cruz/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Botafogo,Avenida%20Oswaldo%20Cruz,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EBotafogo,-22.9398472,-43.175138&transacao=Venda&tipo=Im%C3%B3vel%20usado&areaMinima=211&areaMaxima=500&tipoUnidade=Residencial,Apartamento': 6
}    

In [35]:
for (url, max_pag) in oswaldo_cruz.items():
    for pagina in range(1,max_pag+1):
        print(url + '&pagina=' + str(pagina))

https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/avenida-oswaldo-cruz/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Botafogo,Avenida%20Oswaldo%20Cruz,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EBotafogo,-22.9398472,-43.175138&transacao=Venda&areaMaxima=100&areaMinima=0&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado&pagina=1
https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/avenida-oswaldo-cruz/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Botafogo,Avenida%20Oswaldo%20Cruz,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EBotafogo,-22.9398472,-43.175138&transacao=Venda&areaMaxima=100&areaMinima=0&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado&pagina=2
https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/avenida-oswaldo-cruz/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Botafogo,Avenida%20Oswaldo%20Cruz,,,BR%3ERio%20de%20Janeiro%3ENULL%3ER

In [116]:
#criando a função

def get_infos(cards_all):
    _df = pd.DataFrame(columns=['Address', 'Area', 'Dorms', 'Baths', 'Parking Slots', 'Condominio', 'IPTU', 'Price'])
    for each_card in cards_all:
            card_features = each_card.find("div", attrs={"class": "simple-card__actions"})
            card_price = each_card.find("p", attrs={"class": "simple-card__price js-price heading-regular heading-regular__bolder align-left"})

            #address
            try:
                address = card_features.find("p", attrs={"class": "color-dark text-regular simple-card__address"}).get_text()
            except:
                address = 'NULL'
            
            #area
            try:
                area_text_and_image = card_features.ul.find("li", attrs={"class": "feature__item text-small js-areas"})
                area_only_text = area_text_and_image.find_all("span")[1]
                area = area_only_text.get_text()
            except:
                area = 'NULL'
                
            #dorms
            try:
                dorms_text_and_image = card_features.ul.find("li", attrs={"class": "feature__item text-small js-bedrooms"})
                dorms_only_text = dorms_text_and_image.find_all("span")[1]
                dorms = dorms_only_text.get_text()
            except:
                dorms = 'NULL'
                
            #baths
            try:
                baths_text_and_image = card_features.ul.find("li", attrs={"class": "feature__item text-small js-bathrooms"})
                baths_only_text = baths_text_and_image.find_all("span")[1]
                baths = baths_only_text.get_text()
            except:
                baths = 'NULL'
            
            #parking_slots
            try:
                parking_text_and_image = card_features.ul.find("li", attrs={"class": "feature__item text-small js-parking-spaces"})
                parking_only_text = parking_text_and_image.find_all("span")[1]
                parking_slots = parking_only_text.get_text()
            except:
                parking_slots = 'NULL'
                
            #condomínio
            try:
                condominio_text = each_card.find("li", attrs={"class": "card-price__item condominium text-regular"})
                condominio = condominio_text.find("span", attrs={"class": "card-price__value"}).get_text()
            except:
                condominio = 'NULL'
                
            #condomínio
            try:
                iptu_text = each_card.find("li", attrs={"class": "card-price__item iptu text-regular"})
                iptu = iptu_text.find("span", attrs={"class": "card-price__value"}).get_text()
            except:
                iptu = 'NULL'
            
            
            #price
            try:
                price = card_price.strong.get_text()
            except:
                price = 'NULL'

            #print(address, area, baths, dorms, price)
            _df = _df.append({'Address': address, 'Area': area, 'Dorms': dorms, 'Baths': baths, 'Parking Slots': parking_slots, 'Condominio': condominio, 'IPTU': iptu, 'Price': price}, ignore_index=True)
        
    return _df

In [ ]:
#'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/avenida-oswaldo-cruz/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Botafogo,Avenida%20Oswaldo%20Cruz,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EBotafogo,-22.9398472,-43.175138&transacao=Venda&areaMaxima=100&areaMinima=0&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado&pagina=2'
df = pd.DataFrame(columns=['Address', 'Area', 'Dorms', 'Baths', 'Parking Slots', 'Condominio', 'IPTU', 'Price'])

df_new = get_infos(cards_all)

df = pd.concat([df, df_new], axis=0)
df

In [166]:
oswaldo_cruz = {
'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/avenida-oswaldo-cruz/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Botafogo,Avenida%20Oswaldo%20Cruz,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EBotafogo,-22.9398472,-43.175138&transacao=Venda&areaMaxima=100&areaMinima=0&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 6,
'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/avenida-oswaldo-cruz/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Botafogo,Avenida%20Oswaldo%20Cruz,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EBotafogo,-22.9398472,-43.175138&transacao=Venda&areaMaxima=130&areaMinima=101&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 4,
'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/avenida-oswaldo-cruz/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Botafogo,Avenida%20Oswaldo%20Cruz,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EBotafogo,-22.9398472,-43.175138&transacao=Venda&areaMaxima=210&areaMinima=131&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 9,
'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/avenida-oswaldo-cruz/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Botafogo,Avenida%20Oswaldo%20Cruz,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EBotafogo,-22.9398472,-43.175138&transacao=Venda&tipo=Im%C3%B3vel%20usado&areaMinima=211&areaMaxima=500&tipoUnidade=Residencial,Apartamento': 6
}    

rui_barbosa = {
'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/avenida-rui-barbosa/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Avenida%20Rui%20Barbosa,,street,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.941972,-43.173507&transacao=Venda&areaMaxima=190&areaMinima=0&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 8,
'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/avenida-rui-barbosa/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Avenida%20Rui%20Barbosa,,street,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.941972,-43.173507&transacao=Venda&areaMaxima=300&areaMinima=191&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 9,
'https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/avenida-rui-barbosa/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Avenida%20Rui%20Barbosa,,street,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.941972,-43.173507&transacao=Venda&areaMinima=301&tipo=Im%C3%B3vel%20usado': 8
}

honorio_barros = {
'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/rua-honorio-de-barros/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Rua%20Hon%C3%B3rio%20de%20Barros,,street,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.9394176,-43.1758625&transacao=Venda&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 3    
}

barao_icarai = {
    'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/rua-barao-de-icarai/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Rua%20Bar%C3%A3o%20de%20Icara%C3%AD,,street,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.938868,-43.175398&transacao=Venda&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 4
}

senador_vergueiro = {
    'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/rua-senador-vergueiro/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Rua%20Senador%20Vergueiro,,street,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.936943,-43.175787&transacao=Venda&areaMaxima=75&areaMinima=0&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 8,
    'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/rua-senador-vergueiro/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Rua%20Senador%20Vergueiro,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.936943,-43.175787&transacao=Venda&areaMaxima=95&areaMinima=76&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 9,
    'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/rua-senador-vergueiro/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Rua%20Senador%20Vergueiro,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.936943,-43.175787&transacao=Venda&areaMaxima=150&areaMinima=96&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 9,
    'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/rua-senador-vergueiro/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Rua%20Senador%20Vergueiro,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.936943,-43.175787&transacao=Venda&areaMinima=151&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 8
}

marques_abrantes = {
    'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/rua-marques-de-abrantes/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Rua%20Marqu%C3%AAs%20de%20Abrantes,,street,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.93711,-43.177533&transacao=Venda&areaMaxima=85&areaMinima=0&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 9,
    'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/rua-marques-de-abrantes/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Rua%20Marqu%C3%AAs%20de%20Abrantes,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.93711,-43.177533&transacao=Venda&areaMaxima=103&areaMinima=86&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 9,
    'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/rua-marques-de-abrantes/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Rua%20Marqu%C3%AAs%20de%20Abrantes,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.93711,-43.177533&transacao=Venda&areaMinima=104&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 10
}

senador_euzebio = {
    'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/rua-senador-euzebio/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Rua%20Senador%20Euzebio,,street,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.9378206,-43.1749561&transacao=Venda&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 4
}

princesa_januaria = {
    'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/rua-princesa-januaria/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Rua%20Princesa%20Januaria,,street,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.9383876,-43.1751276&transacao=Venda&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 2
}

marques_parana = {
    'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/rua-marques-de-parana/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Rua%20Marqu%C3%AAs%20de%20Paran%C3%A1,,street,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.938123,-43.177086&transacao=Venda&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 2
}

cruz_lima = {
    'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/rua-cruz-lima/?transacao=Venda&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado&onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Rua%20Cruz%20Lima,,street,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.936769,-43.175169': 4
}

fernando_osorio = {
    'https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/rua-fernando-osorio/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Rua%20Fernando%20Os%C3%B3rio,,street,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.93641,-43.176243&transacao=Venda&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado': 3
}


In [169]:
df = pd.DataFrame(columns=['Address', 'Area', 'Dorms', 'Baths', 'Parking Slots', 'Condominio', 'IPTU', 'Price'])

for (url, max_pag) in fernando_osorio.items():
    for pagina in range(1,max_pag+1):
        url_pag = url + '&pagina=' + str(pagina)
        
        print('Scrapping from: ', url_pag)
        
        
        request_body = scraper.get(url_pag).text

        soup = bs(request_body)

        cards_all = soup.find_all("div", attrs={"class": "box--display-flex box--flex-column gutter-top-double gutter-left-double gutter-right-double gutter-bottom-double simple-card__box"})
        
        
        df_new = get_infos(cards_all)
        df = pd.concat([df, df_new], axis=0)
        time.sleep(5)

Scrapping from:  https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/rua-fernando-osorio/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Rua%20Fernando%20Os%C3%B3rio,,street,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.93641,-43.176243&transacao=Venda&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado&pagina=1
Scrapping from:  https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/rua-fernando-osorio/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Rua%20Fernando%20Os%C3%B3rio,,street,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,-22.93641,-43.176243&transacao=Venda&tipoUnidade=Residencial,Apartamento&tipo=Im%C3%B3vel%20usado&pagina=2
Scrapping from:  https://www.zapimoveis.com.br/venda/apartamentos/rj+rio-de-janeiro/rua-fernando-osorio/?onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,Rua%20Fernando%20Os%C3%B3rio,,street,BR%3ERio%20de%20

In [172]:
df.Price.value_counts()

\n    R$ 650.000\n    /venda      8
\n    R$ 930.000\n    /venda      6
\n    R$ 850.000\n    /venda      5
\n    Sob consulta\n              4
\n    R$ 950.000\n    /venda      3
\n    R$ 680.000\n    /venda      2
\n    R$ 700.000\n    /venda      1
\n    R$ 1.855.000\n    /venda    1
\n    R$ 980.000\n    /venda      1
\n    R$ 1.204.900\n    /venda    1
\n    R$ 740.000\n    /venda      1
\n    R$ 1.194.000\n    /venda    1
\n    R$ 748.000\n    /venda      1
\n    R$ 750.000\n    /venda      1
\n    R$ 890.000\n    /venda      1
\n    R$ 970.000\n    /venda      1
\n    R$ 878.000\n    /venda      1
\n    R$ 880.000\n    /venda      1
\n    R$ 1.665.000\n    /venda    1
\n    R$ 920.000\n    /venda      1
\n    R$ 960.000\n    /venda      1
\n    R$ 1.100.000\n    /venda    1
\n    R$ 860.000\n    /venda      1
\n    R$ 630.000\n    /venda      1
\n    R$ 895.000\n    /venda      1
\n    R$ 940.000\n    /venda      1
\n    R$ 768.000\n    /venda      1
\n    R$ 690.000\n    /venda

In [171]:
df.to_csv('fernando_osorio.csv', index=False)